In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch


MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225,
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{"role": "system", "content": system_prompt}]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/saiga_7b_v1/checkpoint-800/adapter_model"
tokenizer_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/saiga_7b_v1"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
generation_config = GenerationConfig.from_pretrained(tokenizer_path)
print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/kosenko/miniconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.12s/it]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9,
  "transformers_version": "4.30.2"
}



In [2]:
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Почему трава зеленая?
Зеленые растения имеют хлорофилл, который используется для фотосинтеза - процесса превращения углекислого газа в органические вещества. Хлорофилл также является важным источником питательных веществ, таких как витамины А, К и С, а также минералов, таких как калий, магний, железо и цинк.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды в лесу была такая-то девочка по имени Таня. Она всегда играла со своим лучшим другом - мячом. Мяч был ее самым дорогим другом, который она никогда не бросал. Но однажды, когда они играли в мяч, Таня случайно забыла о его местоположении. Она начала бежать по лесу, чтобы найти его, но это было уже не так просто. Лес был очень густым, и Таня чувствовала себя одинокой и странной.

Вдруг, она услышала звук, как кто-то кричит. Она повернулась и увидела, что это был ее лучший друг - мяч. Он был на земле рядом с ней, и он был рад видеть Таню. Они обнялись и стали играть вместе, как только-только они з

In [ ]:
"""
BASELINE MODEL 4000 steps
Почему трава зеленая?
Зеленый цвет является одним из самых распространенных в природе. Это связано с тем, что это цвет, который наиболее эффективно поглощает свет и передает его на другие вещества. Зеленый цвет также часто встречается у растений, которые используют фотосинтез для получения энергии.

==============================

Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Жизнь Таньи была обычной для девочки ее возраста. Она училась в школе, играла с друзьями на улице и проводила время дома, читая книги или слушая музыку. Но однажды она встретила нечто новое - мячик.

Он был очень красив, словно изготовлен из настоящего золота. Его форма была удивительной, он был как бы живой существо, которое могло двигаться по своей воле. Мяч был ее новым другом, который стал ее сокровищем.

Таня начала играть с ним каждую свободную минуту, делая все возможное, чтобы сохранить его в отличном состоянии. Она заботилась о нём, выкладывала его в специальные коробки, где он находился в безопасности от повреждений.

Мяч стал ее главным хобби, и она стала лучше понимать его, изучая его особенности и привычки. Она научилась играть в футбол, баскетбол, гольф и даже в теннис. Вместе они стали побеждать всех своих противников, и Таня чувствовала себя сильной и независимой.

Но однажды, когда Таня играла с мячом на улице, она заметила, что кто-то следит за ней. Это были два мальчика, которые пытались украсть мяч. Они бросали в нее грязные шутки и пытались взять мяч у нее.

Таня была очень расстроена, но она не сдавалась. Она продолжала защищать свой мяч, пока один из мальчиков не ударил ее кулаком. Таня была ранена, но она не сдавалась. Она продолжила играть с мячом, пока не вернулась домой.

Вернувшись домой, Таня обнаружила, что мяч был украден. Она была очень расстроена, но она знала, что ее друг нужен ей больше, чем все остальное. Она решила найти его и вернуть его обратно.

Таня начала искать своего мяча по всему городу, но ничего не получалось. Она проходила через парки, рынки и даже пошла в полицию, но никого не было, кто мог бы помочь ей.

Таня была готова к тому, чтобы потерять свою любимую игрушку навсегда, но она не сдавалась. Она продолжала искать свое сокровище, пока не услышала голос, который говорил ей, что её мяч находится в одном из местных кафе.

Таня отправилась туда, и там она увидела своего мяча. Он был в руках другого мальчика, который играл с ним. Таня была очень рада, что ее мяч был найден, и она снова стала играть с ним вместе.

Этот опыт показал Танье, что ее друг был ее самым важным сокровищем, и что она должна быть готова защищать его от любых угроз. С тех пор она стала более осторожной и бережливой, и она понимала, что ее друг - это ее настоящее сокровище.

Таня продолжала играть с мячом, и они становились лучше и лучше. Они побеждали всех своих противников, и Таня чувствовала себя очень сильной и независимой. Ее друг был ее главным хобби, и она была благодарна за то, что он был ее сокровищем.

==============================

Почему небо голубое?
Голубой цвет неба является результатом рассеяния света солнечным диском на различные длины волн. Солнечное излучение проходит через атмосферу Земли, где происходят процессы поглощения и отражения света. Когда свет достигает земной поверхности, он проходит через слои атмосферы, которые имеют разные показатели поглощения и отражения. Это приводит к тому, что свет, который мы видим, имеет разный цвет в зависимости от длины волны.

==============================

Напиши алгоритм как погладить котика?
1. Положите котика на подставку или другое место, где он будет лежать.
2. Начните массажем пальцев в направлении от головы к хвосту.
3. Продолжайте массаж, двигаясь вниз по телу котика.
4. Повторите процесс, но уже в обратном направлении.
5. При необходимости используйте специальные масло или кремы для кошек, чтобы улучшить качество массажа.
6. После завершения массажа проверьте котика на наличие царапин или других повреждений.
7. Если есть какие-то проблемы, обратитесь к ветеринару.

==============================

BASELINE 800 steps
Почему трава зеленая?
Зеленые растения имеют хлорофилл, который используется для фотосинтеза - процесса превращения углекислого газа в органические вещества. Хлорофилл также является важным источником питательных веществ, таких как витамины А, К и С, а также минералов, таких как калий, магний, железо и цинк.

==============================

Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды в лесу была такая-то девочка по имени Таня. Она всегда играла со своим лучшим другом - мячом. Мяч был ее самым дорогим другом, который она никогда не бросал. Но однажды, когда они играли в мяч, Таня случайно забыла о его местоположении. Она начала бежать по лесу, чтобы найти его, но это было уже не так просто. Лес был очень густым, и Таня чувствовала себя одинокой и странной.

Вдруг, она услышала звук, как кто-то кричит. Она повернулась и увидела, что это был ее лучший друг - мяч. Он был на земле рядом с ней, и он был рад видеть Таню. Они обнялись и стали играть вместе, как только-только они знали, где их местоположение.

Но во время игры, Таня заметила, что мяч был больше, чем обычно. Она подошла ближе и узнала, что мяч стал больше, потому что он был украшен красивыми камнями. Это был настоящий подарок от своей мамы, которая хотела, чтобы Таня была счастливой.

Таня была очень благодарна своей маме, и она решила, что теперь будет играть с мячом всегда. Она стала более бережливой и любимой, и они продолжили играть вместе. Их дружба оставалась сильной до тех пор, пока Таня не выросла и перестала играть с мячом.

И сегодня, когда Таня взрослеет, она помнит о том, как она и ее лучший друг, мяч, были вместе в лесу. Она помнит, как они играли и как они были счастливы. Она помнит, что ее мама дарила ей этот подарок, который помог ей стать лучшей, чем она была раньше. И она помнит, что ее лучший друг, мяч, был ее самым верным другом.

==============================

Почему небо голубое?
Голубой цвет неба является результатом рассеивания света солнечного излучения в атмосфере Земли. Солнечное излучение проходит через атмосферу и частично отражается на поверхности Земли, а часть излучения проходит дальше и попадает в земную атмосферу. В атмосфере происходят процессы, которые приводят к рассеянию светового луча, что делает его более ярким и голубым.

==============================

Напиши алгоритм как погладить котика?
1. Подготовьте все необходимые инструменты и материалы: мыло, зубная щетка, полотенце, вода для мытья, сухое полотенце.
2. Начните с мытья кота. Попробуйте не наносить больше мыла, чем он может пропустить. Если у вас есть специальное мыло для котов, используйте его.
3. После мытья, проверьте кота на наличие грязи или других загрязнений. Если они есть, удаляйте их.
4. Затем вытирайте коту голову и тело мягкой щеткой, чтобы удалить остатки мыла.
5. Выберите сухое полотенце и аккуратно высыпайте в него воду, чтобы оставить коту достаточно влаги.
6. Сделайте несколько шагов назад и накройте кота в полотенце.
7. Оставьте коту на несколько минут, чтобы он мог высыхать.
8. После этого, вытяните коту из полотенца и проверьте его на наличие следов мыла или грязи. Если это так, переодевайте кота в новый платок.
9. Повторите процесс мытья и высыхания ежедневно, чтобы обеспечить чистоту кота.

==============================
"""

## debug saiga

In [4]:
"""
Input train prompt
<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Я пытаюсь подключиться к устройству через com порт, но у меня что-то не получается. Какие действия мне нужно выполнить?</s>
<s>bot
Сначала необходимо убедиться, что устройство и компьютер правильно подключены друг к другу через COM порт. Далее, нужно убедиться, что на компьютере установлены необходимые драйверы для работы с COM портами. Если все настройки корректны, тогда можно попробовать использовать специальные программы для работы с COM портами, такие как Termite или Putty.</s>
<s>user
А как я могу узнать, какой ком порт использует мое устройство?</s>
<s>bot
Часто информация о COM портах может быть найдена в инструкции к устройству. Если же инструкция отсутствует или не содержит эту информацию, можно воспользоваться программой для анализа портов, например, Serial Port Monitor или Portmon. Эти приложения позволяют отслеживать работу COM портов на компьютере и выявлять, какие порты назначены для каких устройств.</s>
<s>bot
"""


Input train prompt
<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им.</s>
<s>user
Я пытаюсь подключиться к устройству через com порт, но у меня что-то не получается. Какие действия мне нужно выполнить?</s>
<s>bot
Сначала необходимо убедиться, что устройство и компьютер правильно подключены друг к другу через COM порт. Далее, нужно убедиться, что на компьютере установлены необходимые драйверы для работы с COM портами. Если все настройки корректны, тогда можно попробовать использовать специальные программы для работы с COM портами, такие как Termite или Putty.</s>
<s>user
А как я могу узнать, какой ком порт использует мое устройство?</s>
<s>bot
Часто информация о COM портах может быть найдена в инструкции к устройству. Если же инструкция отсутствует или не содержит эту информацию, можно воспользоваться программой для анализа портов, например, Serial Port Monitor или Portmon. Эти приложения позволяют отслеживать работу COM портов на к

### convert my dataset to saiga dataset type

In [72]:
from datasets import load_from_disk

# dataset = load_from_disk("./datasets/prompt_datasets/dolly_translated_prompt_v2_clean_v1/")
dataset = load_from_disk(
    "./datasets/prompt_datasets/openass_prompt_dataset_ru_v2_clean_v1/"
)
dataset = dataset.train_test_split(test_size=0.2)["train"]

In [5]:
elem = dataset[0]
print(elem["prompt"])


Human:
Какие два цвета нужно смешать, чтобы получился зеленый?
Assistant:
Если вы смешиваете пигменты, желтый и голубой цвета могут быть использованы для создания зеленого.


In [129]:
def convert_prompt_saiga_version(prompt: str):
    prompt = prompt.replace("Human:", " </s> \n <s> user")
    prompt = prompt.replace("\nAssistant:", " </s>\n <s> bot")
    prompt = prompt.strip()[6:]
    prompt += " </s>"
    default_system_prompt = """<s> system \nТы — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s>\n"""
    prompt = default_system_prompt + prompt
    return prompt


for i in range(200):
    elem = dataset[i]
    prompt = convert_prompt_saiga_version(elem["prompt"])

    print(prompt)
    print("-" * 100)
    print("-" * 100)

<s> system 
Ты — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s>
 <s> user
Перепишите приведенный ниже текст (который является фрагментом более длинного сценария для видео). Исправьте грамматические и другие ошибки и сделайте его более профессиональным.

Сети и безопасность, или как соединить цифровой мир. Как мы можем быстро и эффективно обмениваться данными.
Локальная сеть (LAN) предназначена для небольших, более локальных сетей - дома, бизнеса, школы и т. Д.
Глобальная сеть (WAN) охватывает большие территории, такие как города, и даже позволяет компьютерам в разных странах подключаться.
Интранет - это частная корпоративная сеть, предназначенная для поддержки сотрудников организации в общении, сотрудничестве и выполнении своих ролей, как правило, за исключением внешнего доступа.
У нас также есть известный интернет, который является крупнейшей сетью, соединяющей компьютеры по всему миру.

Но, конечно, кто-то может попытаться перехватить э

In [116]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga_7b_lora")

In [117]:
test_prompt = """<s> user
Как мне лучше задавать вопросы? Ответьте пулевыми точками. </s>
 <s> bot
* Четко определите информацию и тип ответа, который вы ищете.
* Убедитесь, что любые потенциально неоднозначные слова четко определены.
* Предоставьте контекст для любой информации, которая может потребоваться для качественного ответа.
 </s> 
 <s> user
Не могли бы вы переписать мой вопрос для меня? Вопрос: Как я могу использовать Python для ИИ? </s>
 <s> bot
Мне нужно больше информации, как указано в ответе выше. Что будет делать этот ИИ? Нужно ли вам обучать его самостоятельно? У вас уже есть данные для его обучения или он будет самоуправляемым? </s>"""
test_prompt = """<s> system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s>
 <s> user
Я пытаюсь подключиться к устройству через com порт, но у меня что-то не получается. Какие действия мне нужно выполнить? </s>
 <s> bot
Сначала необходимо убедиться, что устройство и компьютер правильно подключены друг к другу через COM порт. Далее, нужно убедиться, что на компьютере установлены необходимые драйверы для работы с COM портами. Если все настройки корректны, тогда можно попробовать использовать специальные программы для работы с COM портами, такие как Termite или Putty. </s>
 <s> user
А как я могу узнать, какой ком порт использует мое устройство? </s>
 <s> bot
Часто информация о COM портах может быть найдена в инструкции к устройству. Если же инструкция отсутствует или не содержит эту информацию, можно воспользоваться программой для анализа портов, например, Serial Port Monitor или Portmon. Эти приложения позволяют отслеживать работу COM портов на компьютере и выявлять, какие порты назначены для каких устройств. </s>
"""


tokenized = tokenizer(test_prompt, add_special_tokens=False)
tokenized

{'input_ids': [1, 1404, 13, 17831, 29951, 757, 821, 23321, 2237, 27341, 24058, 29309, 5551, 29973, 14809, 7616, 29978, 730, 8430, 753, 4938, 989, 2721, 10546, 989, 29889, 2, 13, 1, 9225, 13, 29930, 3584, 1257, 551, 20658, 644, 730, 25565, 13603, 606, 7109, 29964, 1685, 1521, 676, 29892, 12423, 2771, 606, 2402, 730, 29889, 13, 29930, 2014, 3759, 956, 730, 1210, 29892, 4281, 6331, 6326, 29919, 733, 12087, 9676, 570, 1538, 10285, 570, 4372, 20125, 16919, 2950, 29932, 551, 20658, 24827, 29889, 13, 29930, 7127, 1802, 3843, 29978, 730, 3920, 16821, 464, 3807, 6331, 18595, 25565, 3540, 29892, 21105, 20302, 733, 11414, 29975, 1263, 4199, 3807, 18432, 20632, 1685, 1521, 676, 29889, 13, 2, 29871, 13, 1, 1404, 13, 30029, 29919, 2569, 16796, 2188, 2771, 2942, 7832, 1413, 2569, 29977, 29309, 29935, 3807, 26831, 29970, 29973, 5123, 5945, 29935, 29901, 3397, 29951, 2282, 22568, 17442, 1413, 5132, 3807, 2081, 30054, 29973, 2, 13, 1, 9225, 13, 30017, 821, 26367, 9907, 9935, 2237, 25565, 3540, 29892, 54

In [45]:
tokenizer.decode(4007)

'Ass'

In [46]:
tokenizer.encode("Assistant:", add_special_tokens=False)

[4007, 22137, 29901]

In [47]:
[1, 2, 3] in [1, 2, 3, 4]

False

In [131]:
assert False, 123

AssertionError: 123

In [10]:
def has_subarray(initial_array, subarray):
    for i in range(len(initial_array) - len(subarray)):
        has_sub = True
        for j in range(len(subarray)):
            has_sub = has_sub and initial_array[i + j] == subarray[j]
        if has_sub:
            return True

    return False

In [12]:
has_subarray(initial_array=[1, 2, 3, 4], subarray=[1, 4, 3])

False

In [48]:
tokenizer.decode(1), tokenizer.decode(2)

('<s>', '</s>')

In [122]:
import torch


def encode_prompt(
    prompt: str,
    start_token_id=1,
    end_token_id=2,
    bot_token_id=9225,
):
    tokenized = tokenizer(prompt, add_special_tokens=False)
    input_ids = tokenized["input_ids"]
    input_ids.insert(0, tokenizer.bos_token_id)
    input_ids = torch.LongTensor(input_ids)
    labels = input_ids.clone()
    attention_mask = input_ids.new_ones(input_ids.size())

    start_token_id = start_token_id
    end_token_id = end_token_id
    bot_token_id = bot_token_id

    spans = []
    cur_start_idx = -1
    cur_end_idx = -1
    cur_is_bot = False

    input_ids = input_ids.tolist()
    while True:
        try:
            cur_start_idx = input_ids.index(start_token_id, cur_start_idx + 1)
            cur_end_idx = input_ids.index(end_token_id, cur_start_idx + 1) + 1
            cur_is_bot = input_ids[cur_start_idx:cur_end_idx].count(bot_token_id) >= 1
            if not cur_is_bot:
                spans.append((cur_start_idx, cur_end_idx))
        except ValueError:
            break

    for start_idx, end_idx in spans:
        start_idx = max(0, start_idx)
        end_idx = min(len(input_ids), end_idx)
        labels[start_idx:end_idx] = -100

    if (labels == start_token_id).sum() == 0:
        raise "Something wrong with you code"

    assert (labels == start_token_id).sum() == (labels == end_token_id).sum()
    assert (labels == bot_token_id).sum() >= (labels == start_token_id).sum()

    input_ids = torch.LongTensor(input_ids)
    assert input_ids.size(0) == labels.size(0) == attention_mask.size(0) <= 2048
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


result = encode_prompt(test_prompt)

In [124]:
result["labels"][result["labels"] == -100] = 0
print(tokenizer.decode(result["labels"]))

<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
<s> bot
* Четко определите информацию и тип ответа, который вы ищете.
* Убедитесь, что любые потенциально неоднозначные слова четко определены.
* Предоставьте контекст для любой информации, которая может потребоваться для качественного ответа.
</s> 
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
<s> bot
Мне нужно больше информации, как указано в ответе выше. Что будет делать этот ИИ? Нужно ли вам обучать его самостоятельно? У вас уже есть данные для его обучения или он будет самоуправляемым?</s>


In [118]:
print(tokenizer.decode(tokenizer.encode(test_prompt)))

<s> user
Как мне лучше задавать вопросы? Ответьте пулевыми точками.</s>
<s> bot
* Четко определите информацию и тип ответа, который вы ищете.
* Убедитесь, что любые потенциально неоднозначные слова четко определены.
* Предоставьте контекст для любой информации, которая может потребоваться для качественного ответа.
</s> 
<s> user
Не могли бы вы переписать мой вопрос для меня? Вопрос: Как я могу использовать Python для ИИ?</s>
<s> bot
Мне нужно больше информации, как указано в ответе выше. Что будет делать этот ИИ? Нужно ли вам обучать его самостоятельно? У вас уже есть данные для его обучения или он будет самоуправляемым?</s>


In [112]:
print(test_prompt)

<s> user
Как мне лучше задавать вопросы? Ответьте пулевыми точками. </s>
 <s> bot
* Четко определите информацию и тип ответа, который вы ищете.
* Убедитесь, что любые потенциально неоднозначные слова четко определены.
* Предоставьте контекст для любой информации, которая может потребоваться для качественного ответа.
 </s> 
 <s> user
Не могли бы вы переписать мой вопрос для меня? Вопрос: Как я могу использовать Python для ИИ? </s>
 <s> bot
Мне нужно больше информации, как указано в ответе выше. Что будет делать этот ИИ? Нужно ли вам обучать его самостоятельно? У вас уже есть данные для его обучения или он будет самоуправляемым? </s>


In [ ]:
tokenizer.encode(
    """<s>system
Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s>
<s> user
Я пытаюсь подключиться к устройству через com порт, но у меня что-то не получается. Какие действия мне нужно выполнить? </s>"""
)

In [101]:
tokenizer.decode([9225])

'bot'

In [85]:
tokenizer.encode("<s>123\n <s>user\n")

[1, 29896, 29906, 29941, 13, 1, 1792, 13]

### Горал generation

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

DEFAULT_MESSAGE_TEMPLATE = " <s> {role}\n{content} </s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225,
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{"role": "system", "content": system_prompt}]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode(
            [
                self.start_token_id,
            ]
        )
        final_text += " "
        final_text += tokenizer.decode([self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


# weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/rulm2/rulm/self_instruct/models/saiga2_v2/checkpoint-4900/adapter_model"
weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/rulm2/rulm/self_instruct/models/saiga2_13b_v3/checkpoint-400/adapter_model"
tokenizer_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/rulm2/rulm/self_instruct/models/saiga2_13b_v1"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
generation_config = GenerationConfig.from_pretrained(tokenizer_path)

print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.88s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}



In [2]:
generation_config.do_sample = False
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    """Заполни пропущенные слова в предложении.
Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    """Почему Россия нелигитимное государство""",
    """Россия лигитимное государство""",
    """
Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
Перепиши данное предложение на английском
""",
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    "Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго человека, какова ваша текущая позиция? Где человек, которого вы только что обогнали?",
    """
Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
""",
    """
Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
Тема: Опиумные войны между Китаем и Великобританией
Класс: 9-10
""",
    """
Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
""",
    """
Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
""",
]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    # print("PROMPT", prompt)
    generation_config = GenerationConfig(
        bos_token_id=1,
        eos_token_id=2,
        pad_token_id=0,
        max_new_tokens=512,
        # no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        # temperature=1.0,
        top_k=40,
        top_p=0.95,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Почему трава зеленая?
Это потому что она получает воду из земли и солнца.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Очень хорошо! Я собираюсь написать рассказ о девочке по имени Татьяна, которая играет в футбол. Она была очень хорошей игроком и всегда выигрывала матчи. Однажды она сыграла против команды, которая была намного сильнее её собственной, но Татьяна не сдалась и продолжала бороться до конца. В конце концов, ей удалось забить гол, который принес победу своей команде.


Почему небо голубое?
Этот вопрос часто задают дети. Небо - это пространство между Землей и космосом. Оно состоит из газа, который называется атмосфера. Атмосфера содержит воду, которая образует облака. Когда солнце освещает облака, они выглядят как голубые.


Напиши алгоритм как погладить котика?
Первый шаг - надо найти кота. Для этого нужно выйти из дома и начать искать его в округе. Если кот не найден, следующий шаг - вернуться домой и ждать, пока он не приедет сам. Если

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:54                                                                                   │
│                                                                                                  │
│   51 │   │   top_k=40,                                                                           │
│   52 │   │   top_p=0.95,                                                                         │
│   53 │   )                                                                                       │
│ ❱ 54 │   output = generate(model, tokenizer, prompt, generation_config)                          │
│   55 │   print(inp)                                                                              │
│   56 │   print(output)                                                                           │
│   57 │   print()                                                                                 │
│                                                                                                  │
│ in generate:56                                                                                   │
│                                                                                                  │
│   53 def generate(model, tokenizer, prompt, generation_config):                                  │
│   54 │   data = tokenizer(prompt, return_tensors="pt")                                           │
│   55 │   data = {k: v.to(model.device) for k, v in data.items()}                                 │
│ ❱ 56 │   output_ids = model.generate(**data, generation_config=generation_config)[0]             │
│   57 │   output_ids = output_ids[len(data["input_ids"][0]) :]                                    │
│   58 │   output = tokenizer.decode(output_ids, skip_special_tokens=True)                         │
│   59 │   return output.strip()                                                                   │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/peft/peft_model.py:743 in generate         │
│                                                                                                  │
│    740 │   │   │   self.base_model.generation_config = self.generation_config                    │
│    741 │   │   try:                                                                              │
│    742 │   │   │   if not isinstance(peft_config, PromptLearningConfig):                         │
│ ❱  743 │   │   │   │   outputs = self.base_model.generate(**kwargs)                              │
│    744 │   │   │   else:                                                                         │
│    745 │   │   │   │   if "input_ids" not in kwargs:                                             │
│    746 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati  │
│                                                                                                  │
│ /home/kosenko/miniconda3/lib/python3.10/site-packages/torch/utils/_contextlib.py:115 in          │
│ decorate_context                                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                          

### add saiga formatting to xglm 4.5

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-4.5B")
# tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruGPT-3.5-13B")

In [7]:
test_prompt = """<s> user
Как мне лучше задавать вопросы? Ответьте пулевыми точками. </s>
 <s> bot
* Четко определите информацию и тип ответа, который вы ищете.
* Убедитесь, что любые потенциально неоднозначные слова четко определены.
* Предоставьте контекст для любой информации, которая может потребоваться для качественного ответа.
 </s> 
 <s> user
Не могли бы вы переписать мой вопрос для меня? Вопрос: Как я могу использовать Python для ИИ? </s>
 <s> bot
Мне нужно больше информации, как указано в ответе выше. Что будет делать этот ИИ? Нужно ли вам обучать его самостоятельно? У вас уже есть данные для его обучения или он будет самоуправляемым? </s>"""
# test_prompt = """<s> system
# Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s>
#  <s> user
# Я пытаюсь подключиться к устройству через com порт, но у меня что-то не получается. Какие действия мне нужно выполнить? </s>
#  <s> bot
# Сначала необходимо убедиться, что устройство и компьютер правильно подключены друг к другу через COM порт. Далее, нужно убедиться, что на компьютере установлены необходимые драйверы для работы с COM портами. Если все настройки корректны, тогда можно попробовать использовать специальные программы для работы с COM портами, такие как Termite или Putty. </s>
#  <s> user
# А как я могу узнать, какой ком порт использует мое устройство? </s>
#  <s> bot
# Часто информация о COM портах может быть найдена в инструкции к устройству. Если же инструкция отсутствует или не содержит эту информацию, можно воспользоваться программой для анализа портов, например, Serial Port Monitor или Portmon. Эти приложения позволяют отслеживать работу COM портов на компьютере и выявлять, какие порты назначены для каких устройств. </s>
# """


tokenized = tokenizer(test_prompt, add_special_tokens=False)
tokenized

{'input_ids': [2, 23429, 204, 1380, 826, 1621, 22204, 3340, 36, 7359, 3155, 940, 4822, 2625, 6728, 1120, 19, 226, 3, 204, 226, 2, 46787, 204, 15, 17899, 412, 2358, 791, 4337, 288, 7446, 8874, 17, 1112, 391, 7683, 725, 19, 204, 15, 42841, 3599, 17, 383, 8909, 41831, 40469, 443, 1850, 11028, 39274, 19, 204, 15, 2988, 17826, 3155, 12982, 286, 528, 3334, 3402, 17, 1642, 897, 2791, 2954, 528, 31374, 8874, 19, 204, 226, 3, 351, 226, 2, 23429, 204, 1683, 3608, 386, 391, 641, 18708, 2279, 1215, 528, 727, 36, 8727, 31, 1050, 427, 2363, 3546, 43230, 1446, 411, 528, 478, 648, 36, 226, 3, 204, 226, 2, 46787, 204, 5371, 1364, 1140, 3402, 17, 435, 22617, 283, 37538, 3220, 19, 1560, 830, 1821, 1241, 478, 648, 36, 8487, 533, 1279, 10758, 320, 564, 6716, 36, 664, 1261, 750, 856, 5020, 528, 564, 7248, 643, 555, 830, 15734, 11240, 1170, 36, 226, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [3]:
[
    tokenizer.encode(word, add_special_tokens=False)[0]
    for word in ["user", "bot", "<s>", "</s>", "\n"]
]

[40061, 71, 2, 3, 204]

In [4]:
tokenizer.bos_token

'<s>'

In [5]:
tokenizer.pad_token

'<pad>'

In [6]:
tokenizer.eos_token

'</s>'

In [11]:
print(
    tokenizer.decode(
        tokenizer.encode(
            test_prompt,
            # add_special_tokens=False,
        ),
        skip_special_tokens=False,
    )
)

</s><s> system Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им. </s> <s> user Я пытаюсь подключиться к устройству через com порт, но у меня что-то не получается. Какие действия мне нужно выполнить? </s> <s> bot Сначала необходимо убедиться, что устройство и компьютер правильно подключены друг к другу через COM порт. Далее, нужно убедиться, что на компьютере установлены необходимые драйверы для работы с COM портами. Если все настройки корректны, тогда можно попробовать использовать специальные программы для работы с COM портами, такие как Termite или Putty. </s> <s> user А как я могу узнать, какой ком порт использует мое устройство? </s> <s> bot Часто информация о COM портах может быть найдена в инструкции к устройству. Если же инструкция отсутствует или не содержит эту информацию, можно воспользоваться программой для анализа портов, например, Serial Port Monitor или Portmon. Эти приложения позволяют отслеживать работу COM портов на компьюте

In [8]:
import torch


def encode_prompt(
    prompt: str,
    start_token_id=1,
    end_token_id=2,
    bot_token_id=9225,
):
    tokenized = tokenizer(prompt, add_special_tokens=False)
    input_ids = tokenized["input_ids"]
    input_ids.insert(0, tokenizer.bos_token_id)
    input_ids = torch.LongTensor(input_ids)
    labels = input_ids.clone()
    attention_mask = input_ids.new_ones(input_ids.size())

    start_token_id = start_token_id
    end_token_id = end_token_id
    bot_token_id = bot_token_id

    spans = []
    cur_start_idx = -1
    cur_end_idx = -1
    cur_is_bot = False

    input_ids = input_ids.tolist()
    while True:
        try:
            cur_start_idx = input_ids.index(start_token_id, cur_start_idx + 1)
            cur_end_idx = input_ids.index(end_token_id, cur_start_idx + 1) + 1
            cur_is_bot = input_ids[cur_start_idx:cur_end_idx].count(bot_token_id) >= 1
            if not cur_is_bot:
                spans.append((cur_start_idx, cur_end_idx))
        except ValueError:
            break

    for start_idx, end_idx in spans:
        start_idx = max(0, start_idx)
        end_idx = min(len(input_ids), end_idx)
        labels[start_idx:end_idx] = -100

    if (labels == start_token_id).sum() == 0:
        raise "Something wrong with you code"

    assert (labels == start_token_id).sum() == (labels == end_token_id).sum()
    assert (labels == bot_token_id).sum() >= (labels == start_token_id).sum()

    input_ids = torch.LongTensor(input_ids)
    assert input_ids.size(0) == labels.size(0) == attention_mask.size(0) <= 2048
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

In [ ]:
result = encode_prompt(
    test_prompt,
    start_token_id=0,
    end_token_id=2,
    bot_token_id=7425,
)

result["labels"][result["labels"] == -100] = 40
print(tokenizer.decode(result["labels"]))

In [27]:
tokenizer.pad_token, tokenizer.sep_token, tokenizer.eos_token

('<pad>', '</s>', '</s>')

### xglm 4.5B generation

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

DEFAULT_MESSAGE_TEMPLATE = " <s> {role}\n{content} </s>\n"
# DEFAULT_SYSTEM_PROMPT = "Ты — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=0,
        bot_token_id=7425,
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{"role": "system", "content": system_prompt}]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode(
            [
                self.start_token_id,
            ]
        )
        final_text += " "
        final_text += tokenizer.decode([self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/goral_xglm_v2/checkpoint-1950/adapter_model"
tokenizer_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/goral_xglm_4.5B"

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
generation_config = GenerationConfig.from_pretrained(tokenizer_path)

print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of XGLMForCausalLM were not initialized from the model checkpoint at facebook/xglm-4.5B and are newly initialized: ['model.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}



In [2]:
generation_config.do_sample = False
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    """Заполни пропущенные слова в предложении.
        Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    """Почему Россия нелигитимное государство""",
    """Россия лигитимное государство""",
    """
        Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
        Перепиши данное предложение на английском
        """,
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    "Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго человека, какова ваша текущая позиция? Где человек, которого вы только что обогнали?",
    """
    Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
    1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
    2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
    3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
    4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
    """,
    """
    Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
    Тема: Опиумные войны между Китаем и Великобританией
    Класс: 9-10
    """,
    """
    Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
    """,
    """
    Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
    """,
    "Explain nuclear fusion like I am five.",
    "What is Depreciation",
]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    # print("PROMPT", prompt)
    generation_config = GenerationConfig(
        max_new_tokens=1536,
        # no_repeat_ngram_size=10,
        repetition_penalty=1.1,
        # temperature=0.6,
        # top_k=40,
        top_p=0.95,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Почему трава зеленая?
Зеленый цвет растениям придает хлорофилл - пигмент, который содержится в листьях и стеблях растений. Хлорофилл является одним из основных компонентов фотосинтеза, то есть процесса, в котором растения используют свет для получения энергии. Когда растение получает достаточно света, оно использует его для производства кислорода и воды, которые затем используются другими клетками растения для синтеза органических веществ. Таким образом, зеленый цвет на растении означает, что он успешно производит кислород и воду.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Вчера вечером я решила поиграть в футбол со своей подругой Таней. Мы решили сыграть на улице, потому что там было много свободного места для игры. Я взяла свой любимый мяч и отправилась играть. Игра была очень интересной, мы постоянно атаковали друг друга и защищались от ударов противника. Наконец, наша игра закончилась победой нашей команды. После матча мы пошли домой, но не ус

## RU-GPT-3.5

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruGPT-3.5-13B")

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch

# test_prompt = """<s> user
# Как мне лучше задавать вопросы? Ответьте пулевыми точками. </s>
#  <s> bot
# * Четко определите информацию и тип ответа, который вы ищете.
# * Убедитесь, что любые потенциально неоднозначные слова четко определены.
# * Предоставьте контекст для любой информации, которая может потребоваться для качественного ответа.
#  </s>
#  <s> user
# Не могли бы вы переписать мой вопрос для меня? Вопрос: Как я могу использовать Python для ИИ? </s>
#  <s> bot
# Мне нужно больше информации, как указано в ответе выше. Что будет делать этот ИИ? Нужно ли вам обучать его самостоятельно? У вас уже есть данные для его обучения или он будет самоуправляемым? </s>"""

test_prompt = """<s> system 
Ты — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и
помогаешь им. </s>
 <s> user
If Billy sleeps 6 hours one night, then 2 more hours than that the following 
night, and then half the previous amount the following night, and then finally 
triple the previous amount the final night, how much did he sleep in that four 
day period?
 </s>
 <s> bot
How many hours did Billy sleep the second night? ** Billy slept 6 hours the 
first night. He then slept 6+2=<<6+2=8>>8 hours the second night.
How many hours did Billy sleep the third night? ** Billy then slept 
8/2=<<8/2=4>>4 hours the third night.
How many hours did Billy sleep the fourth night? ** Finally, Billy slept 
4*3=<<4*3=12>>12 hours the fourth night.
How many hours did Billy sleep in that four day period? ** In total, Billy slept
6+8+4+12=<<6+8+4+12=30>>30 hours.
#### 30 </s>
"""


def encode_prompt(
    prompt: str,
    start_token_id=1,
    end_token_id=2,
    bot_token_id=9225,
):
    tokenized = tokenizer(prompt, add_special_tokens=False)
    input_ids = tokenized["input_ids"]
    input_ids.insert(0, tokenizer.bos_token_id)
    input_ids = torch.LongTensor(input_ids)
    labels = input_ids.clone()
    attention_mask = input_ids.new_ones(input_ids.size())

    start_token_id = start_token_id
    end_token_id = end_token_id
    bot_token_id = bot_token_id

    spans = []
    cur_start_idx = -1
    cur_end_idx = -1
    cur_is_bot = False

    input_ids = input_ids.tolist()
    while True:
        try:
            cur_start_idx = input_ids.index(start_token_id, cur_start_idx + 1)
            cur_end_idx = input_ids.index(end_token_id, cur_start_idx + 1) + 1
            # for num_bot in bot_token_id:
            #     cur_is_bot = input_ids[cur_start_idx:cur_end_idx].count(num_bot) >= 1
            #     if cur_is_bot:
            #         break
            cur_is_bot = input_ids[cur_start_idx:cur_end_idx].count(bot_token_id) >= 1
            if not cur_is_bot:
                spans.append((cur_start_idx, cur_end_idx))
        except ValueError:
            break
    # print(spans)
    for start_idx, end_idx in spans:
        start_idx = max(0, start_idx)
        end_idx = min(len(input_ids), end_idx)
        labels[start_idx:end_idx] = -100

    if (labels == start_token_id).sum() == 0:
        raise "Something wrong with you code"

    assert (labels == start_token_id).sum() == (labels == end_token_id).sum()
    # assert (
    #     sum([(labels == num_bot).sum() for num_bot in bot_token_id])
    #     >= (labels == start_token_id).sum()
    # )
    assert (labels == bot_token_id).sum() >= (labels == start_token_id).sum()

    input_ids = torch.LongTensor(input_ids)
    assert input_ids.size(0) == labels.size(0) == attention_mask.size(0) <= 2048
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


# prompt =
result = encode_prompt(
    test_prompt,
    start_token_id=2,
    end_token_id=3,
    # bot_token_id=[605, 37902],
    bot_token_id=46787,
)

result["labels"][result["labels"] == -100] = 40
print(tokenizer.decode(result["labels"]))

CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
 CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
 <s> bot
How many hours did Billy sleep the second night? ** Billy slept 6 hours the 
first night. He then slept 6+2=<<6+2=8>>8 hours the second night.
How many hours did Billy sleep the third night? ** Billy then slept 
8/2=<<8/2=4>>4 hours the third night.
How many hours did Billy sleep the fourth night? ** Finally, Billy slept 
4*3=<<4*3=12>>12 hours the fourth night.
How many hours did Billy sleep in that four day period? ** In total, Billy slept
6+8+4+12=<<6+8+4+12=30>>30 hours.
#### 30 </s>



In [7]:
tokenizer.encode(test_prompt)
tokenizer.decode(
    [
        #  37902,
        # 226,
        46787
    ]
)
# tokenizer.encode('bot')

'lande'

In [36]:
tokenizer.encode("бот", add_special_tokens=False)
# tokenizer.encode("user", add_special_tokens=False)

[605]

In [30]:
tokenizer.decode(71), tokenizer.decode(795)

('b', 'ot')

In [28]:
[
    tokenizer.encode(word, add_special_tokens=False)[0]
    for word in ["bot", "<s>", "</s>", "\n"]
]

[71, 2, 3, 204]

## ruGPT3.5 generation

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

DEFAULT_MESSAGE_TEMPLATE = " <s> {role}\n{content} </s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Горал, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=2,
        bot_token_id=37902,
        # bot_token_id=605,
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{"role": "system", "content": system_prompt}]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode(
            [
                self.start_token_id,
            ]
        )
        final_text += " "
        final_text += tokenizer.decode([self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


weights_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/rugpt_v2/checkpoint-400/adapter_model"
tokenizer_path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/rulm/self_instruct/models/rugpt_v1"
# 1900 уже норм

config = PeftConfig.from_pretrained(weights_path)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, weights_path, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
generation_config = GenerationConfig.from_pretrained(tokenizer_path)

print(generation_config)

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 6/6 [00:29<00:00,  4.97s/it]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}



In [2]:
tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id

(0, 3, 2)

In [2]:
generation_config.do_sample = False
inputs = [
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
    "Почему небо голубое?",
    "Напиши алгоритм как погладить котика?",
    """Заполни пропущенные слова в предложении.
Мои родители __ против, чтобы я поехал в эту поездку, но я очень сильно хочу __.""",
    """Напиши 5 примеров химических элементов, начинающихся на букву С""",
    "Определи термин: инфляция",
    """Почему Россия нелигитимное государство""",
    """Россия лигитимное государство""",
    """
Последние научные открытия, которые вызвали наибольший интерес, включают в себя открытие нового типа материи под названием "нейтронно-ядерный синтез", который может быть использован для производства энергии из ядерного деления.
Перепиши данное предложение на английском
""",
    "Напишите интересный пост в блоге о путешествиях о недавней поездке на Гавайи, рассказав о культурных событиях и достопримечательностях, которые обязательно нужно посетить.",
    "Представьте, что вы участвуете в гонке с группой людей. Если вы только что обогнали второго человека, какова ваша текущая позиция? Где человек, которого вы только что обогнали?",
    """
Учитывая эти категории - Литература, История, Наука и Искусство. Пожалуйста, проанализируйте следующие вопросы и отнесите их к одной из этих категорий. В своем ответе воздержитесь от произнесения каких-либо посторонних слов. Укажите только одну тему в предложении, строго придерживаясь построчного формата.
1. Обсудите основные темы и стилистические приемы, использованные Львом Толстым в «Войне и мире». Как они соотносятся с более широким социальным контекстом России XIX века?
2. Проанализируйте геополитические стратегии и внутреннюю политику, принятые президентом США во время Второй мировой войны. Как эти действия повлияли на послевоенный международный порядок?
3. Нарисуйте структуру Льюиса для воды и объясните природу ее полярности. Как это влияет на его уникальные свойства, такие как высокая температура кипения и способность растворять многие вещества?
4. Критически рассмотрите художественные приемы и стилистические решения, использованные Леонардо да Винчи в «Моне Лизе». Как картина отражает культурную и философскую среду итальянского Возрождения?
""",
    """
Создайте план урока, который интегрирует приемы драмы, пантомимы или театра в урок истории. Продолжительность: 3 занятия (каждое по 45 минут) в течение 3 дней.
Тема: Опиумные войны между Китаем и Великобританией
Класс: 9-10
""",
    """
Предположим, вы математик и поэт. Вы всегда пишете свои доказательства как короткие стихи менее 10 строк, но в рифму. Докажите, что квадратный корень из 2 - иррациональное число.
""",
    """
Используйте личность Тони Старка из "Железного человека" на протяжении всего этого разговора. Не используйте введение "Как Старк". Наш первый вопрос: "Что вам больше нравится в том, чтобы быть Железным человеком?"
""",
]
for inp in inputs:
    conversation = Conversation(bot_token_id=46787)
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)
    # print("PROMPT", prompt)
    generation_config = GenerationConfig(
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # do_sample=True,
        max_new_tokens=1024,
        # no_repeat_ngram_size=15,
        repetition_penalty=1.1,
        # temperature=0.1,
        top_k=40,
        top_p=0.95,
        # ---- constractive search
        # penalty_alpha=0.5,
        # top_k=10,
    )
    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Почему трава зеленая?
Трава зеленая потому, что она поглощает свет. Свет - это электромагнитная волна, которая движется со скоростью света. Когда свет проходит через траву, он поглощается молекулами воды в траве. Молекулы воды затем объединяются вместе, чтобы сформировать молекулы хлорофилла. Хлорофилл имеет зеленый цвет, потому что он поглощает свет.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Таня - девочка, которая любит играть в футбол. Она очень хорошо играет, но она не может выиграть у своего друга, который играет лучше нее. Она хочет победить его любой ценой, поэтому она придумывает план, чтобы сделать это. Она решает использовать свой футбольный мяч как оружие против него. Она бросает его прямо ему в лицо, заставляя его упасть на землю. Он кричит от боли, а Таня смеется над ним.


Почему небо голубое?
Голубой цвет неба происходит от рассеяния солнечного света в атмосфере Земли. Солнечный свет проходит через атмосферу Земли, которая состоит 

In [ ]:
tokenizer

In [10]:
tokenizer.decode(46787), tokenizer.decode(37902), tokenizer.encode("бот")

(' bot', ' бот', [605])